In [14]:
import os
import openai
import json

**Initialize Groq/OpenAI client**

In [15]:
os.environ["GROQ_API_KEY"] = "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"

client = openai.OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ.get("GROQ_API_KEY")
)

**JSON Schema**

In [16]:
REQUIRED_FIELDS = ["name", "email", "phone", "location", "age"]

**Function: Extract user info**

In [26]:
def extract_info(chat_text):
    """
    Extract structured information from chat text using Groq API.
    Ensures output is valid JSON.
    """
    REQUIRED_FIELDS = ["name", "email", "phone", "location", "age"]

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",   # 👈 use available model
        messages=[
            {"role": "system", "content": "You are an information extractor. Only reply in valid JSON."},
            {"role": "user", "content": f"Extract the following fields from this chat:\n"
                                        f"{REQUIRED_FIELDS}\n\nChat:\n{chat_text}"}
        ],
        temperature=0,                  # deterministic output for JSON
        max_completion_tokens=300,
        top_p=1,
        stream=False,
        response_format={"type": "json_object"}  # 👈 forces JSON output
    )

    # Parse JSON safely
    try:
        extracted = json.loads(response.choices[0].message.content)
    except json.JSONDecodeError as e:
        print("Error parsing JSON:", e)
        extracted = {"error": "Failed to parse JSON"}

    return extracted

**Sample chats**

In [27]:
sample_chats = [
    "Hi, my name is Alice Johnson. You can reach me at alice@example.com or call 555-1234. I live in New York and I’m 29 years old.",
    "Hello, I’m Bob, 35 years old from Los Angeles. My email is bob@xyz.com, phone number is 987-6543.",
    "This is Charlie, 42 years old, based in Chicago. Contact me at charlie123@mail.com or 111-222-3333."
]

**Function: Validate schema**

In [28]:
def validate_schema(data):
    """
    Validate extracted JSON against required schema.
    Returns True if all fields exist, False otherwise.
    """
    return all(field in data for field in REQUIRED_FIELDS)

**Run extraction on sample chats**

In [29]:
for idx, chat in enumerate(sample_chats):
    print(f"\n--- Chat Sample {idx+1} ---")
    info = extract_info(chat)
    print("Extracted Info:", info)
    print("Valid Schema:", validate_schema(info))


--- Chat Sample 1 ---
Extracted Info: {'name': 'Alice Johnson', 'email': 'alice@example.com', 'phone': '555-1234', 'location': 'New York', 'age': 29}
Valid Schema: True

--- Chat Sample 2 ---
Extracted Info: {'name': 'Bob', 'email': 'bob@xyz.com', 'phone': '987-6543', 'location': 'Los Angeles', 'age': 35}
Valid Schema: True

--- Chat Sample 3 ---
Extracted Info: {'name': 'Charlie', 'email': 'charlie123@mail.com', 'phone': '111-222-3333', 'location': 'Chicago', 'age': 42}
Valid Schema: True
